# Project: 멋진 챗봇 만들기

지난 노드에서 챗봇과 번역기는 같은 집안이라고 했던 말을 기억하시나요?  
앞서 배운 Seq2seq번역기와 Transfomer번역기에 적용할 수도 있겠지만, 이번 노드에서 배운 번역기 성능 측정법을 챗봇에도 적용해 봅시다.

Step 1. 데이터 다운로드
아래 링크에서 ChatbotData.csv 를 다운로드해 챗봇 훈련 데이터를 확보합니다. csv 파일을 읽는 데에는 pandas 라이브러리가 적합합니다. 읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수에 나눠서 저장하세요!

## Step 1. 데이터 다운로드

In [1]:
# udo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
# python3 -m pip install --upgrade pip
# python3 -m pip install konlpy
# sudo apt-get install curl git
# bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [2]:
import re
import pandas as pd
import os
import numpy as np
import random
import gensim
import tensorflow as tf
from konlpy.tag import Mecab
from tqdm import tqdm

In [3]:
file_path = os.getenv("HOME") + '/aiffel/transformer_chatbot/data/ChatbotData .csv'

dataset = pd.read_csv(file_path)
dataset.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [4]:
questions, answers = dataset['Q'], dataset['A']

In [5]:
questions

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

## Step 2. 데이터 정제
1. 영문자의 경우, 모두 소문자로 변환합니다.  
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.  
문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 우리가 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않아도 괜찮습니다.

In [6]:
def preprocess_sentence(dataset):
    dataset = dataset.lower() # 영문자 소문자로 변환
    re.sub(r"[^a-zA-Zㅏ-ㅣㄱ-ㅎ-가-힣0-9,.?!]+", " ", dataset)
    return dataset

## Step 3. 데이터 토큰화

토큰화에는 KoNLPy의 mecab 클래스를 사용합니다.  
1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.  
2. 데이터를 앞서 정의한 `preprocess_sentence()` 함수로 정제하고, 토큰화합니다.  
3. 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 `mecab.morphs` 함수를 전달하면 됩니다.  
4. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외됩니다.  
5. 중복되는 문장은 데이터에서 제외합니다. 소스:타겟 쌍을 비교하지 않고 소스는 소스대로, 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!  
      
구현한 함수를 활용하여 `questions`와 `answers`를 각각 `que_corpus,` `ans_corpus`에 토큰화하여 저장합니다.

In [7]:
def build_corpus(src_sentence, tgt_sentence):
    tokenized_src, tokenized_len_src = [], []
    tokenized_tgt, tokenized_len_tgt = [], []
    for src, tgt in zip(src_sentence, tgt_sentence):
        tokenizer = Mecab()
        mecab_src = tokenizer.morphs(preprocess_sentence(src))
        mecab_tgt = tokenizer.morphs(preprocess_sentence(tgt))
        
        tokenized_src.append(mecab_src)
        tokenized_tgt.append(mecab_tgt)
        
        tokenized_len_src.append(len(mecab_src))
        tokenized_len_tgt.append(len(mecab_tgt))
        
    total_length = tokenized_len_src + tokenized_len_tgt
    
    max_len = np.max(total_length)
    mean_len = np.mean(total_length)
    mid_len = np.median([mean_len, max_len])
    print(f'mid_len : {mid_len}')
        
    src_corpus, tgt_corpus = [], []
    for s, t in zip(tokenized_src, tokenized_tgt):
        if len(s) < mid_len and s not in tokenized_src:
            tokenized_src.append(s)
            
        if len(t) < mid_len and t not in tokenized_tgt:
            tokenized_tgt.append(t)
            
    return tokenized_src, tokenized_tgt

In [8]:
que_corpus, ans_corpus = build_corpus(questions, answers)

mid_len : 23.852046857819506


In [9]:
print(que_corpus[:5], "\n",ans_corpus[:5])

[['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'], ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'], ['ppl', '심하', '네']] 
 [['하루', '가', '또', '가', '네요', '.'], ['위로', '해', '드립니다', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['여행', '은', '언제나', '좋', '죠', '.'], ['눈살', '이', '찌푸려', '지', '죠', '.']]


In [10]:
len(que_corpus), len(ans_corpus)

(11823, 11823)

## Step 4. Augmentation
우리에게 주어진 데이터는 1만 개가량으로 적은 편에 속합니다. 이럴 때에 사용할 수 있는 테크닉을 배웠으니 활용해 봐야겠죠? Lexical Substitution을 실제로 적용해 보도록 하겠습니다.  
  
링크를 참고하여 한국어로 사전 훈련된 Embedding 모델을 다운로드합니다. `Korean(w)` 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 `Korean(w)`를 찾아 다운로드하고, `ko.bin` 파일을 얻으세요!  
  
다운로드한 모델을 활용해 데이터를 Augmentation 하세요! 앞서 정의한 `lexical_sub()` 함수를 참고하면 도움이 많이 될 겁니다.  
  
Augmentation된 `que_corpus` 와 원본 `ans_corpus` 가 병렬을 이루도록, 이후엔 반대로 원본 `que_corpus` 와 Augmentation된 `ans_corpus` 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배 가량으로 늘어나도록 합니다.

In [11]:
# 4.0.X 버전부터는 gensim 안에서 vocab 코드가 패키지 안에서 사라지기 때문에 ko.bin을 로드하기
# 위해서는 다운그레이드를 해줘야 함
# pip install --upgrade gensim==3.8.3

In [12]:
ko_bin_path = os.getenv('HOME') + '/aiffel/NLPGD/GD6/ko.bin'
word2vec = gensim.models.Word2Vec.load(ko_bin_path)

In [13]:
def lexical_sub(sentence, word2vec):
    try: 
        _from = random.choice(sentence) # sentence에서 하나의 형태소를 무작위로 고름
        _to = word2vec.most_similar(_from)[0][0] # 가장 비슷한 단어를 추출
    except: # sentence 문장이 word2vec에 없으면 그냥 sentence를 반환
        return sentence
    
    res = []
    for x in sentence:
        if x is _from: res.append(_to)
        else: res.append(x)
            
    return res

In [14]:
a = '싫'
b = word2vec.most_similar(a)
b

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('힘들', 0.6567603349685669),
 ('귀찮', 0.634676456451416),
 ('일쑤', 0.6083942651748657),
 ('편하', 0.6059445738792419),
 ('꺼리', 0.5969125628471375),
 ('길들이', 0.5821220874786377),
 ('어렵', 0.5785283446311951),
 ('풀어쓰', 0.570091187953949),
 ('아깝', 0.5619203448295593),
 ('웃기', 0.5485275983810425)]

In [15]:
que_corpus[1]

['1', '지망', '학교', '떨어졌', '어']

In [16]:
arg_que_corpus = [lexical_sub(x, word2vec) for x in que_corpus]
arg_ans_corpus = [lexical_sub(x, word2vec) for x in ans_corpus]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


In [17]:
for i in range(5):
    print("Q:", " ".join(que_corpus[i]), "/" , " ".join(arg_que_corpus[i]))
    print("A:", " ".join(ans_corpus[i]), "/" , " ".join(arg_ans_corpus[i]))

Q: 12 시 땡 ! / 12 시가 땡 !
A: 하루 가 또 가 네요 . / 하루 가 또 놀드 네요 .
Q: 1 지망 학교 떨어졌 어 / 1 지망 학교 떨어졌 어
A: 위로 해 드립니다 . / 위로 해의 드립니다 .
Q: 3 박 4 일 놀 러 가 고 싶 다 / 3 박 4 일 놀 러 가 고 싶 으며
A: 여행 은 언제나 좋 죠 . / 여행 은 언제나 좋 죠 는데
Q: 3 박 4 일 정도 놀 러 가 고 싶 다 / 3 박 4 일 만큼 놀 러 가 고 싶 다
A: 여행 은 언제나 좋 죠 . / 여행 은 언제나 좋 죠 는데
Q: ppl 심하 네 / ppl 강하 네
A: 눈살 이 찌푸려 지 죠 . / 눈살 이 찌푸려 지 죠 .


In [18]:
que_corpus = que_corpus + arg_que_corpus + que_corpus
ans_corpus = ans_corpus + ans_corpus + arg_ans_corpus

In [19]:
len(que_corpus), len(ans_corpus)

(35469, 35469)

### Step 5. 데이터 벡터화
타겟 데이터인 `ans_corpus` 에 `<start>` 토큰과 `<end>` 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다.

In [20]:
a = ['i', 'love', 'you']
a = ['<start>'] + a + ['<end>']
a

['<start>', 'i', 'love', 'you', '<end>']

In [21]:
def add_tokens(sentence):
    added_tokens = []
    for list in sentence:
        temp_list = ['<start>'] + list + ['<end>']

        added_tokens.append(temp_list)
    
    return added_tokens

ans_corpus = add_tokens(ans_corpus)

In [22]:
ans_corpus[:5]

[['<start>', '하루', '가', '또', '가', '네요', '.', '<end>'],
 ['<start>', '위로', '해', '드립니다', '.', '<end>'],
 ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>'],
 ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>'],
 ['<start>', '눈살', '이', '찌푸려', '지', '죠', '.', '<end>']]

챗봇 훈련 데이터의 가장 큰 특징 중 하나라고 하자면 바로 소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것이겠죠. 앞서 배운 것처럼 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있습니다.  
  
특수 토큰을 더함으로써 `ans_corpus` 또한 완성이 되었으니, `que_corpus` 와 결합하여 전체 데이터에 대한 단어 사전을 구축하고 벡터화하여 `enc_train` 과 `dec_train` 을 얻으세요!

In [23]:
all_data = que_corpus + ans_corpus

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters=' ', oov_token='<unk>')
tokenizer.fit_on_texts(all_data)
tensor = tokenizer.texts_to_sequences(all_data)
tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

In [24]:
tensor[:5]

array([[2513,  183, 3605,   91,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 291, 3606,  560, 1043,   11,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 312, 2222,  549,   60,  224,  249,    9,   12,   37,   32,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 312, 2222,  549,   60,  455,  224,  249,    9,   12,   37,   32,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    

In [53]:
VOCAB_SIZE = len(tokenizer.index_word)
VOCAB_SIZE

7985

In [54]:
for i in range(1,6):
    print(i, ':', tokenizer.index_word[i])

1 : <unk>
2 : .
3 : <start>
4 : <end>
5 : 이


합친 `que_corpus`, `ans_corpus`를 다시 분리해줍니다.

In [27]:
print(tensor.shape, len(all_data))

(70938, 42) 70938


In [28]:
enc_train, dec_train = tensor[:len(que_corpus)], tensor[len(ans_corpus):]

In [29]:
enc_train.shape, dec_train.shape

((35469, 42), (35469, 42))

### Step 6. 훈련하기

앞서 번역 모델을 훈련하며 정의한 Transformer 를 그대로 사용하시면 됩니다! 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다.

**Positional Encoding**  
$PE(pos, 2i) = Sin(pos/10000^{\frac{2i}{d_{model}}})$  
$PE(pos, 2i+1) = Cos(pos/10000^{\frac{2i+1}{d_{model}}})$

In [58]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

**Generate Padding Mask**

In [59]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

**Multi-head Attention**
$Attention(Q, K, V) = Softmax(\frac{QK^T}{\sqrt{d_k}})V$

In [60]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

**Position-wise Feed Forward Network**

In [61]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

**Encoder Layer**

In [62]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

**Decoder Layer**

In [63]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual
        
        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

**Encoder**

In [64]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

**Decoder**

In [65]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
                            
    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

**Transformer 전체 모델 조립**

In [66]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

**Learning Rate Scheduler**  
트랜스포머 학습률(learning rate)은 고정된 값을 유지하는 것이 아니라 학습 경과에 따라 변하도록 설게하였습니다. 공식은 아래와 같습니다.  
$lrate = d_{model}^{-0.5}*min(step\_num^{-0.5}, step\_num * warmup\_steps^{-1.5})$

In [67]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

**Loss Function 정의**

In [68]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    
    # Masking 되지 않은 입력의 개수로 Scaling하는 과정
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

**Train Step 정의**

In [78]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

**모델 인스턴스 생성**

In [79]:
transformer = Transformer(
    n_layers=6,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=8000,
    tgt_vocab_size=8000,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,)

d_model = 512

**Learning Rate & Optimizer 정의**

In [80]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

**Train**

In [81]:
EPOCHS = 20
BATCH_SIZE = 64

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = train_step(enc_train[idx:idx+BATCH_SIZE],
                                                                     dec_train[idx:idx+BATCH_SIZE],
                                                                     transformer,
                                                                     optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

Epoch 20: 100%|██████████| 555/555 [05:30<00:00,  1.68it/s, Loss 0.2315]


## Step 7. 성능 측정하기
챗봇의 경우, 올바른 대답을 하는지가 중요한 평가 지표입니다. 올바른 답변을 하는지 눈으로 확인할 수 있겠지만, 많은 데이터의 경우는 모든 결과를 확인할 수 없을 것입니다. 주어진 질문에 적절한 답변을 하는지 확인하고, BLEU Score를 계산하는 calculate_bleu() 함수도 적용해 보세요.

In [83]:
examples = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

In [95]:
def evaluate(sentence, model, tokenizer):
    mecab = Mecab()
    sentence = mecab.morphs(preprocess_sentence(sentence))
    sentence = tokenizer.texts_to_sequences(sentence)
    _input = tf.keras.preprocessing.sequence.pad_sequences([sentence],
                                                          maxlen=enc_train.shape[-1],
                                                          padding='post')
    
    ids = []
    output = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = generate_masks(_input, output)
        print(enc_padding_mask.shape, combined_mask.shape, dec_padding_mask.shape)
        
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(_input, output, enc_padding_mask, combined_mask, dec_padding_mask)
        
        predicted_id = tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()
        
        # 숫자를 문자열로 복원
        if tokenizer.word_index['<end>'] == predicted_id:
            result = ' '.join(tokenizer.sequences_to_texts(idx))
            return pieces, result, enc_attns, dec_attns, dec_enc_attns
        
        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)
        
    result = ' '.join(tokenizer.sequences_to_texts(idx))
    
    return places, result, enc_attns, dec_attns, dec_enc_attns

In [96]:
# 번역 생성
def translate(sentence, model, tokenizer):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = evaluate(sentence, model, tokenizer)
    return result

In [97]:
for sen in examples:
    print(f'Q : {sen}')
    print(f'A : {translate(sen, transformer, tokenizer)}')

Q : 지루하다, 놀러가고 싶어.
(1, 1, 1, 42, 1) (1, 1, 1, 42, 42) (1, 1, 1, 1)


InvalidArgumentError: Input to reshape is a tensor with 512 values, but the requested shape has 42 [Op:Reshape]